In [2]:
!pwd

/content


In [3]:
!ls

drive  sample_data


In [4]:
cd drive

/content/drive


In [5]:
!ls

'My Drive'


In [6]:
cd 'My Drive'

/content/drive/My Drive


In [7]:
!ls

'Colab Notebooks'   deep-learning-from-scratch-master   IMG_0001.pdf


In [0]:
#cd 'Colab Notebooks'

In [0]:
#!pwd

In [0]:
#!ls

In [11]:
cd deep-learning-from-scratch-master

/content/drive/My Drive/deep-learning-from-scratch-master


In [12]:
!ls

 ch01   ch05   common	   't10k-images-idx3-ubyte.gz のコピー'
 ch02   ch06   dataset	   't10k-labels-idx1-ubyte.gz のコピー'
 ch03   ch07   LICENSE.md  'train-images-idx3-ubyte.gz のコピー'
 ch04   ch08   README.md   'train-labels-idx1-ubyte.gz のコピー'


In [13]:
cd ch05

/content/drive/My Drive/deep-learning-from-scratch-master/ch05


In [14]:
!pwd

/content/drive/My Drive/deep-learning-from-scratch-master/ch05


In [15]:
!ls

buy_apple_orange.py  gradient_check.py	__pycache__	    two_layer_net.py
buy_apple.py	     layer_naive.py	train_neuralnet.py


In [0]:
class MulLayer:
  def __init__(self):
    self.x = None
    self.y = None
    
  def forward(self, x, y):
    self.x = x
    self.y = y
    out = x * y
    
    return out
    
  def backward(self, dout):
    dx = dout * self.y #xとyをひっくり返す
    dy = dout * self.x
    
    return dx, dy
  

In [17]:
apple = 100
apple_num = 2
tax = 1.1

#layer
mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

#forward
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

print(price)

220.00000000000003


In [18]:
#backward
dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(dapple, dapple_num, dtax)

2.2 110.00000000000001 200


In [0]:
class AddLayer:
  def __init__(self):
    pass
  
  def forward(self, x, y):
    out = x + y
    return out
  def backward(self, dout):
    dx = dout * 1
    dy = dout * 1
    return dx, dy

In [20]:
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

#layer
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

#forward
apple_price = mul_apple_layer.forward(apple, apple_num) #(1)
orange_price = mul_orange_layer.forward(orange, orange_num) #(2)
all_price = add_apple_orange_layer.forward(apple_price, orange_price) #(3)
price = mul_tax_layer.forward(all_price, tax) #(4)

#backward
dprice = 1
dall_price, dtax = mul_tax_layer.backward(dprice) #(4)
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price) #(3)
dorange, dorange_num = mul_orange_layer.backward(dorange_price) #(2)
dapple, dapple_num = mul_apple_layer.backward(dapple_price) #(1)

print(price)
print(dapple_num, dapple, dorange, dorange_num, dtax)

715.0000000000001
110.00000000000001 2.2 3.3000000000000003 165.0 650


In [0]:
class Relu:
  def __init___(self):
    self.mask = None
    
  def forward(self, x):
    self.mask = (x <= 0)
    out = x.copy()
    out[self.mask] = 0
    
    return out
  
  def backward(self, dout):
    dout[self.mask] = 0
    dx = dout
    
    return dx

In [22]:
import numpy as np
x = np.array([[1.0, -0.5], [-2.0, 3.0]])
print(x)

[[ 1.  -0.5]
 [-2.   3. ]]


In [23]:
mask = (x <= 0)
print(mask)

[[False  True]
 [ True False]]


In [0]:
class Sigmoid:
  def __init__(self):
    self.out = None
    
  def forward(self, x):
    out = 1/ (1 + np.exp(-x))
    self.out = out
    
    return out
  
  def backward(self, dout):
    dx = dout * (1.0 - self.out) * self.out
    
    return dx

In [0]:
X = np.random.rand(2) #入力
W = np.random.rand(2, 3) #重み
B = np.random.rand(3) #バイアス

In [26]:
X.shape

(2,)

In [27]:
W.shape

(2, 3)

In [28]:
B.shape

(3,)

In [0]:
Y = np.dot(X, W) + B

In [0]:
X_dot_W = np.array([[0, 0, 0],[10, 10, 10]])
B = np.array([1, 2, 3])

In [31]:
X_dot_W

array([[ 0,  0,  0],
       [10, 10, 10]])

In [32]:
X_dot_W + B

array([[ 1,  2,  3],
       [11, 12, 13]])

In [33]:
dY = np.array([[1, 2, 3],[4, 5, 6]])
dY

array([[1, 2, 3],
       [4, 5, 6]])

In [34]:
dB = np.sum(dY, axis=0)
dB

array([5, 7, 9])

In [0]:
class Affine:
  def __init__(self, W, b):
    self.W = W
    self.b = b
    self.x = None
    self.dW = None
    self.db = None
    
  def forward(self, x):
    self.x = x
    out = np.dot(x, self.W) + self.b
    
    return out
  def backward(self, dout):
    dx = np.dot(dout, self.W.T)
    self.dW = np.dot(self.x.T, dout)
    self.db = np.sum(dout, axis=0)
    
    return dx

In [0]:
class SoftmaxWIthLoss:
  def __init__(self):
    self.loss = None #損失
    self.y = None #softmaxの出力
    self.t = None #教師データ(one-hot vector)
    
  def forward(self, x, t):
    self.t = t
    self.y = softmax(x)
    self.loss = cross_entropy_error(self.y, self.t)
    
    return self.loss
  
  def backward(self, dout=1):
    batch_size = self.t.shape[0]
    dx = (self.y - self.t) / batch_size
    
    return dx

In [0]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from common.layers import *
from common.gradient import numerical_gradient
from collections import OrderedDict

class TwoLayerNet:
  def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
    #重みの初期化
    self.params = {}
    self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
    self.params['b1'] = np.zeros(hidden_size)
    self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
    self.params['b2'] = np.zeros(output_size)
    
    #レイヤの生成
    self.layers = OrderedDict()
    self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
    self.layers['Relu1'] = Relu()
    self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])
    
    self.lastLayer = SoftmaxWithLoss()
    
    def predict(self, x):
      for layer in self.Layers.values():
        x = layer.forward(x)
        
        return(x)
      
    #x:入力データ #t:教師データ
    def loss(self, x, t):
      y = self.predict(x)
      return self.lastLayer.forward(y, t)
    
    def accuracy(self, x, t):
      y = predict(x)
      y = np.argmax(y, axis=1)
      if t.ndim != 1 : t = npargmax(t, axis=1)
        
      accuracy = np.sum(y == t) / float(x.shape[0])
      return accuracy
    
    #x:入力データ #t:教師データ
    def numerical_gradient(self, x, t):
      loss_W = lambda W: self.loss(x, t)
      
      grads = {}
      grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
      grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
      grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
      grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
      
      return grads
    
    def gradient(self, x, t):
      #forward
      self.loss(x,t)
      
      #backward
      dout = 1
      dout = self.lastLayer.backward(dout)
      
      layers = list(selflayers.values())
      layers.reverse()
      for layer in layers:
        dout = layer.backward(dout)
        
        #設定
        grads = {}
        grads['W1'] = self.layers['Affine1'].dW
        grads['b1'] = self.layers['Affine1'].db
        grads['W2'] = self.layers['Affine2'].dW
        grads['b2'] = self.layers['Affine2'].db
        
        return grads

In [38]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from dataset.mnist import load_mnist
from two_layer_net import TwoLayerNet

#データ読み込み
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

x_batch = x_train[:3]
t_batch = t_train[:3]

grad_numerical = network.numerical_gradient(x_batch, t_batch)
grad_backdrop = network.gradient(x_batch, t_batch)

#各重みの絶対誤差の平均を求める
for key in grad_numerical.keys():
  diff = np.average(np.abs(grad_backdrop[key]) - grad_numerical[key])
  print(key + ":" + str(diff))

W1:0.0005095538049283565
b1:0.003080414953734321
W2:0.006482534750188179
b2:0.13988916600780965


In [39]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from dataset.mnist import load_mnist
from two_layer_net import TwoLayerNet

#データ読み込み
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

iters_num = 10000
train_size = x_train.shape[0]
batch_size =100
learning_rate = 0.1
train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iters_num):
  batch_mask = np.random.choice(train_size, batch_size)
  x_batch = x_train[batch_mask]
  t_batch = t_train[batch_mask]
  
  #誤差逆伝播法によって勾配を求める
  grad = network.gradient(x_batch, t_batch)
  
  #更新
  for key in ('W1', 'b1', 'W2', 'b2'):
    network.params[key] -= learning_rate * grad[key]
    
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    if i % iter_per_epoch == 0:
      train_acc = network.accuracy(x_train, t_train)
      test_acc = network.accuracy(x_test, t_test)
      train_acc_list.append(train_acc)
      test_acc_list.append(test_acc)
      print(train_acc, test_acc)

0.16646666666666668 0.1711
0.16676666666666667 0.1714
0.17975 0.185
0.16871666666666665 0.1728
0.9015333333333333 0.9063
0.9015 0.9064
0.9014166666666666 0.9055
0.9014 0.9055
0.9228 0.9254
0.9227666666666666 0.9256
0.92225 0.9254
0.92225 0.9254
0.9368166666666666 0.9356
0.9368333333333333 0.9358
0.9372833333333334 0.9355
0.9373166666666667 0.9356
0.94445 0.9434
0.9444666666666667 0.9434
0.9447166666666666 0.9437
0.9446666666666667 0.9436
0.9504166666666667 0.9486
0.95045 0.9486
0.9502666666666667 0.9476
0.9503166666666667 0.9476
0.9565833333333333 0.9541
0.9566333333333333 0.9542
0.9564333333333334 0.9545
0.9564 0.9545
0.9605 0.9578
0.9605 0.9579
0.9602166666666667 0.957
0.9602166666666667 0.957
0.9623166666666667 0.9591
0.9623333333333334 0.9591
0.9623833333333334 0.9589
0.9623833333333334 0.9589
0.9666 0.9631
0.9666666666666667 0.9631
0.9666 0.9629
0.96655 0.9629
0.9687666666666667 0.9642
0.9688166666666667 0.9643
0.9688 0.964
0.9687833333333333 0.964
0.97085 0.9663
0.970866666666666

In [1]:
#cd /content

/content
